<h1>Import Libraries

In [2]:
import numpy as np
import time
import sys
import os
import random
from skimage import io
from skimage.transform import resize
import pandas as pd
from matplotlib import pyplot as plt
from shutil import copyfile

import shutil
import cv2
import csv
# import tensorflow as tf
# from keras.preprocessing.image import ImageDataGenerator
path = "OID/Dataset/train/Vehicle registration plate"


In [3]:
def setup():
    base_path = 'OID\csv_folder'
    images_boxable_fname = 'train-images-boxable.csv'
    annotations_bbox_fname = 'train-annotations-bbox.csv'
    class_descriptions_fname = 'class-descriptions-boxable.csv'

    #SETTING Dataframe from CSV
    images_boxable = pd.read_csv(os.path.join(base_path, images_boxable_fname))
    # print(images_boxable.head())

    annotations_bbox = pd.read_csv(os.path.join(base_path, annotations_bbox_fname))
    print(annotations_bbox.head())

    class_descriptions = pd.read_csv(os.path.join(base_path, class_descriptions_fname))
    # print(class_descriptions.head())
    
    #Get the Plate images
    plate_pd = class_descriptions[class_descriptions['class']=='Vehicle registration plate']
    label_plate = plate_pd['name'].values[0]
    print(plate_pd)
    #Find plates in annotations
    plate_bbox = annotations_bbox[annotations_bbox['LabelName']==label_plate]
    return plate_bbox

In [4]:
def get_plates(plate_bbox):
    #Amount of images in dataset
    print('There are %d plate in the dataset' %(len(plate_bbox)))
    plate_img_id = plate_bbox['ImageID']

    #Amount of unique images
    plate_img_id = np.unique(plate_img_id)
    print('There are %d plates in the dataset' %(len(plate_img_id)))

    # Shuffle the ids and pick the first 1000 ids
    copy_plate_id = plate_img_id.copy()
    np.random.seed(1)
    np.random.shuffle(copy_plate_id)

    n = 1000
    subplate_img_id = copy_plate_id[:n]
    # print(subplate_img_id)

    #Find images in randomn list
    files = os.listdir(path)
    train_plate = []
    for f in files:
        f = f[0:-4]
        train_plate.append(f)

    #gets the 1000 matched  images
    rand_plates = list(set(train_plate).intersection(subplate_img_id))
    return rand_plates

In [5]:
def plate_dataframe(rand_plates):
    temp_train = []
    for t in rand_plates:
        temp_train.append("/" + t + '.jpg')
    
    # train_dict = {'FilePath','Filename','XMin':[], 'XMax':[], 'YMin':[], 'YMax':[], 'ClassName'}
    train_df = pd.DataFrame(columns=['FilePath','Filename','XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])
    
    #copy files folder images
    for x in temp_train:
        train_path = path + x
        isFile = os.path.isfile(train_path)
        #checks to see if patht exists
        if(isFile):
            newPath = shutil.copy(train_path, "OID/Dataset/train/Vehicle registration plate/images")
            # train_dict['FilePath'] = newPath
            # train_dict['ClassName'] = "Vehicle registration plate"
            train_df = train_df.append({'FilePath': newPath,'Filename': x[1:-4],'ClassName':"Vehicle registration plate"},ignore_index=True)


    #labels folder append info to that
    lfiles = os.listdir(os.path.join(path,"Label"))
    
    for a in lfiles:
        # if( a[0:-4] in train_dict):
        #     print("TRUE")
        #     with open(os.path.join(os.path.join(path,"Label"),a)) as labeled_file:
        #         csv_reader = csv.reader(labeled_file, delimiter=" ")
        #         for row in csv_reader:
        #             train_dict()
        if(train_df['Filename'].str.contains(a[0:-4]).any()):
            with open(os.path.join(os.path.join(path,"Label"),a)) as labeled_file:
                csv_reader = csv.reader(labeled_file, delimiter=" ")
                temp_xmin = [] #left x
                temp_ymax = [] #top Y
                temp_xmax = [] #right x
                temp_ymin = [] #bottom y
                for row in csv_reader:
                    temp_xmin.append(row[3]) #left x
                    temp_ymax.append(row[4]) #top Y
                    temp_xmax.append(row[5]) #right x
                    temp_ymin.append(row[6]) #bottom y
                #get index of file
                index = train_df.index[train_df["Filename"] == a[0:-4]].tolist()
                train_df.at[index[0], 'XMin'] = temp_xmin
                train_df.at[index[0], 'YMax'] = temp_ymax   
                train_df.at[index[0], 'XMax'] = temp_xmax 
                train_df.at[index[0], 'YMin'] = temp_ymin  
    print(len(train_df))
    print(train_df.head())
    return train_df

In [6]:
#plot bounding box
def plot_box(file_img,train_df):
    #train_df = pd.DataFrame(columns=['FilePath','Filename','XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])
    gray_image = cv2.imread(file_img, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(gray_image,cv2.COLOR_BGR2RGB)
    height, width, channel = image.shape
    print(f"Image: {image.shape}")
    cv2.resize(image,(128,128))
    for index in range(len(train_df["XMin"][0])):
        xmin = int(float(train_df['XMin'][0][index]))
        xmax = int(float(train_df['XMax'][0][index]))
        ymin = int(float(train_df['YMin'][0][index]))
        ymax = int(float(train_df['YMax'][0][index]))
        class_name = train_df['ClassName'][0]
        print(f"Coordinates: {xmin,ymin}, {xmax,ymax}")
        cv2.rectangle(image, (xmax,ymax), (xmin,ymin), (255,0,0),1)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image, "Plate", (xmin,ymax-5), font, 1, (0,0,0), 2)
    plt.title((train_df['Filename'][0]) + ".jpg")
    plt.imshow(image)
    plt.axis("off")
    plt.show()


In [7]:
def main():
    plate_bbox = setup()
    # rand_plates = get_plates(plate_bbox)
    # train_df = plate_dataframe(rand_plates)
    
    # #Get one image from train_df
    # file_img = train_df['FilePath'][0] 
    # plot_box(file_img,train_df)

if __name__ == "__main__":
    main()


#Prepare dataset format for faster rcnn code
# Save images to train and test directory
#train 0.8 and test 0.2
# train_path = os.path.join(path, 'train')
# os.mkdir(train_path)
# test_path = os.path.join(path, 'test')
# os.mkdir(test_path)


            ImageID  Source  LabelName  Confidence      XMin      XMax  \
0  000002b66c9c498e  xclick  /m/01g317           1  0.012500  0.195312   
1  000002b66c9c498e  xclick  /m/01g317           1  0.025000  0.276563   
2  000002b66c9c498e  xclick  /m/01g317           1  0.151562  0.310937   
3  000002b66c9c498e  xclick  /m/01g317           1  0.256250  0.429688   
4  000002b66c9c498e  xclick  /m/01g317           1  0.257812  0.346875   

       YMin      YMax  IsOccluded  IsTruncated  IsGroupOf  IsDepiction  \
0  0.148438  0.587500           0            1          0            0   
1  0.714063  0.948438           0            1          0            0   
2  0.198437  0.590625           1            0          0            0   
3  0.651563  0.925000           1            0          0            0   
4  0.235938  0.385938           1            0          0            0   

   IsInside  
0         0  
1         0  
2         0  
3         0  
4         0  
         name             